In [20]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
from config import g_key
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from config import weather_api_key
from scipy.stats import linregress

In [21]:
# Load in csv
vacation_itinerary_df = pd.read_csv("WeatherPy_Challenge_CSV_Files/WeatherPy_vacation.csv")
vacation_itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Russell,US,86.00,broken clouds,32.35,-85.20,Macs
1,3,San Pedro,PH,82.40,overcast clouds,14.35,121.02,OYO 412 Viewfort Venue
2,5,Vila Velha,BR,75.20,clear sky,-20.33,-40.29,Hotel Vitória Palace
3,9,Vaini,TO,78.80,broken clouds,-21.20,-175.20,Keleti Beach Resort
4,11,Adre,TD,79.99,scattered clouds,13.47,22.20,NaN
...,...,...,...,...,...,...,...,...
171,567,La Asuncion,VE,78.66,overcast clouds,11.03,-63.86,Colinas de Matasiete
172,571,Jamiltepec,MX,75.99,broken clouds,16.28,-97.82,Hotel Marys
173,573,Chengde,CN,79.34,overcast clouds,40.97,117.94,Super 8 Hotel
174,579,Weligama,LK,80.10,overcast clouds,5.97,80.42,Weligama Bay Resort


In [27]:
# Create a new dataframe for Alexandria, Rosetta, Asyut, and Esna.
Alexandria_df = vacation_itinerary_df.loc[vacation_itinerary_df["City"]=="Alexandria"]
Rosetta_df = vacation_itinerary_df.loc[vacation_itinerary_df["City"]=="Rosetta"]
Asyut_df = vacation_itinerary_df.loc[vacation_itinerary_df["City"]=="Asyut"]
Esna_df = vacation_itinerary_df.loc[vacation_itinerary_df["City"]=="Esna"]
Esna_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
83,304,Esna,EG,86.0,clear sky,25.29,32.55,Orient Star Dahabyia Isna 05 Nights Each Sunday


In [39]:
import gmaps
import gmaps.datasets
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Latitude-longitude pairs
Alexandria = [31.22, 29.96]
Rosetta = [31.4, 30.42]
Asyut = [27.18, 31.18]
Esna = [25.29, 32.55]

fig = gmaps.figure()
Alex_Rose_Asyu_Esna = gmaps.directions_layer(
        Alexandria, Esna, waypoints=[Rosetta, Asyut],
        travel_mode='DRIVING')
fig.add_layer(Alex_Rose_Asyu_Esna)
fig


#fig = gmaps.figure()
#Alex_Rose_Asyu_Esna = gmaps.directions_layer(Alexandria, Rosetta, Asyut, Esna)
#fig.add_layer(Alex_Rose_Asyu_Esna)
#fig

Figure(layout=FigureLayout(height='420px'))

In [59]:
four_cities_df = pd.concat([Alexandria_df, Rosetta_df, Asyut_df, Esna_df], join="inner")
four_cities_df


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
114,408,Alexandria,EG,75.2,scattered clouds,31.22,29.96,Mediterranean Azur Hotel
55,204,Rosetta,EG,75.2,scattered clouds,31.40,30.42,Green Valley
53,195,Asyut,EG,77.0,clear sky,27.18,31.18,Florence Hotel Assiut
83,304,Esna,EG,86.0,clear sky,25.29,32.55,Orient Star Dahabyia Isna 05 Nights Each Sunday


In [60]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
four_cities_info = [info_box_template.format(**row) for index, row in four_cities_df.iterrows()]

In [68]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = four_cities_df[["Lat", "Lng"]]
max_temp = four_cities_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=True,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=four_cities_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))